In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.chrome.options import Options

In [2]:
df = pd.read_csv('div_info_final.csv')
df

,Unnamed: 0,Announced,Financial Year,Subject,EX Date,Payment Date,Amount,Indicator,Unnamed: 7,Stock Code,Year,Status
0,0,20 Apr 2022,2021-12-31,Interim Dividend,10 May 2022,27 May 2022,0.0260,Currency,View,5250,2021,Y
1,1,15 Apr 2021,2020-12-31,Interim Dividend,05 May 2021,21 May 2021,0.0160,Currency,View,5250,2020,Y
2,2,22 Apr 2020,2019-12-31,Interim Dividend,12 May 2020,28 May 2020,0.0230,Currency,View,5250,2019,Y
3,3,29 Apr 2019,2018-12-31,Interim Dividend,09 May 2019,23 May 2019,0.0240,Currency,View,5250,2018,Y
4,4,01 Dec 2022,2022-12-31,Income Distribution,23 Dec 2022,12 Jan 2023,0.0385,Currency,View,0800EA,2022,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
4001,4001,23 Jan 2019,2018-11-30,Interim Dividend,11 Feb 2019,06 Mar 2019,0.0200,Currency,View,5131,2018,Y
4002,4002,23 Jan 2019,2018-11-30,Special Dividend,11 Feb 2019,06 Mar 2019,0.0200,Currency,View,5131,2018,Y
4003,4003,17 Oct 2018,2018-11-30,Interim Dividend,02 Nov 2018,28 Nov 2018,0.0200,Currency,View,5131,2018,Y
4004,4004,18 Jul 2018,2018-11-30,Second interim dividend,15 Aug 2018,12 Sep 2018,0.0200,Currency,View,5131,2018,Y


In [3]:
stock_code = df['Stock Code'].unique()

In [4]:
len(stock_code)

634

In [5]:
quarter_rep=pd.DataFrame()

driver = webdriver.Chrome()

#driver = webdriver.Chrome()
driver.implicitly_wait(604800)
new_list=[]

stock=['7148']
for s in stock_code:
    try:
        url='https://www.klsescreener.com/v2/stocks/view/'+s
        driver.get(url)
        p= driver.find_element(By.XPATH,'//*[@id="myTab"]/li[3]/a')
        driver.execute_script("arguments[0].click();",p)
        content = driver.page_source
        soup=bs(content)

        # scrap table

        tables = soup.find_all('table')
        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        i = 0
        has_found = False
        while i < len(dfs) and not has_found:
            n = 0
            while n < len(dfs[i].columns):
                if dfs[i].columns[n] == 'ROE':
                    has_found = True
                n += 1
            i += 1
        if has_found:
            df=dfs[i-1]
            df2=df[['EPS','DPS','NTA','Revenue','P/L','Quarter','Q Date','Financial Year','Announced','ROE']]
            df2['Financial Year'] = pd.to_datetime(df['Financial Year'])
            df2['Financial Year']= pd.to_datetime(df2['Financial Year'].dt.strftime('%Y-%m-%d'))
            filtered_df = df2.loc[(df2['Financial Year'] >= '2018-01-01')
                          & (df2['Financial Year'] < '2023-12-31')]
            filtered_df.reset_index(drop=True, inplace=True)
            filtered_df.drop(filtered_df[filtered_df['EPS'] == filtered_df['Revenue']].index, inplace = True)
            filtered_df.reset_index(drop=True, inplace=True)
            filtered_df['Stock Code']=s
            filtered_df['Year'] = filtered_df['Financial Year'].dt.strftime("%Y")
            quarter_rep = quarter_rep.append(filtered_df, ignore_index=True)
            print("Stock code :"+s+" yes")
        else:
            print("Stock code :"+s+" has no  annual report")
    except:
        print("Stock code :"+s+" cannot scrap")
    # Program to generate a random number between 5 to 30
    # importing the random module
    sec = random.randint(3, 10)

    # program scrapping waiting time
    time.sleep(sec)  # Seconds

Stock code :5250 yes
Stock code :0800EA yes
Stock code :7167 yes
Stock code :03028 yes
Stock code :0218 yes
Stock code :1481 yes
Stock code :5281 yes
Stock code :9148 yes
Stock code :7191 yes
Stock code :0181 yes
Stock code :6599 yes
Stock code :5139 yes
Stock code :5185 yes
Stock code :7078 yes
Stock code :0209 yes
Stock code :2658 yes
Stock code :5116 yes
Stock code :2674 yes
Stock code :2488 yes
Stock code :1163 yes
Stock code :03051 yes
Stock code :5269 yes
Stock code :5127 yes
Stock code :5293 yes
Stock code :5120 yes
Stock code :03011 yes
Stock code :1015 yes
Stock code :6351 yes
Stock code :7083 yes
Stock code :6556 yes
Stock code :5568 yes
Stock code :5088 yes
Stock code :7090 yes
Stock code :5015 yes
Stock code :6432 yes
Stock code :7214 yes
Stock code :7722 yes
Stock code :7129 yes
Stock code :0105 yes
Stock code :7162 yes
Stock code :6399 yes
Stock code :8176 yes
Stock code :7048 yes
Stock code :5130 yes
Stock code :7099 yes
Stock code :5302 yes
Stock code :5204 yes
Stock co

In [6]:
quarter_rep

,EPS,DPS,NTA,Revenue,P/L,Quarter,Q Date,Financial Year,Announced,ROE,Stock Code,Year
0,1.21,0.000,0.1106,988.21m,13.66m,3,2022-09-30,2022-12-31,2022-11-29,10.9%,5250,2022
1,2.29,0.000,0.0984,943.67m,25.77m,2,2022-06-30,2022-12-31,2022-08-25,23.3%,5250,2022
2,2.16,0.000,0.1015,839.98m,24.38m,1,2022-03-31,2022-12-31,2022-05-26,21.3%,5250,2022
3,2.60,0.000,0.0799,795.06m,29.24m,4,2021-12-31,2021-12-31,2022-02-24,32.5%,5250,2021
4,0.13,0.000,0.0522,680.22m,1.44m,3,2021-09-30,2021-12-31,2021-11-25,2.5%,5250,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
11760,2.35,2.000,1.2851,39.45m,10.79m,1,2019-02-28,2019-11-30,2019-04-17,1.8%,5131,2019
11761,3.44,4.000,1.2934,45.03m,15.83m,4,2018-11-30,2018-11-30,2019-01-23,2.7%,5131,2018
11762,3.36,2.000,1.2731,51.02m,17.11m,3,2018-08-31,2018-11-30,2018-10-17,2.6%,5131,2018
11763,2.62,2.000,1.2559,44.02m,12.07m,2,2018-05-31,2018-11-30,2018-07-18,2.1%,5131,2018


In [11]:
quarter_rep.to_csv("div_quarter_report.csv")

In [9]:
quarter_rep.sample(20)

,EPS,DPS,NTA,Revenue,P/L,Quarter,Q Date,Financial Year,Announced,ROE,Stock Code,Year
1726,2.97,0.010,0.7100,350.95m,26.26m,1,2022-03-31,2022-12-31,2022-05-25,4.2%,5273,2022
11382,-37.98,0.000,4.2400,83.59m,-24.73m,4,2020-12-31,2020-12-31,2021-02-23,-9.0%,5016,2020
3164,1.28,0.000,0.8900,73.41m,6.83m,1,2017-12-31,2018-09-30,2018-02-26,1.4%,6939,2018
1418,-0.50,0.000,0.2632,8.04m,-898.00k,2,2020-06-30,2020-12-31,2020-08-25,-1.9%,0191,2020
4868,-0.84,0.000,0.4323,276.85m,-17.73m,1,2020-12-31,2021-09-30,2021-02-25,-1.9%,5161,2021
10365,1.48,0.000,1.6154,26.62m,592.00k,3,2021-10-31,2022-01-31,2021-12-30,0.9%,7439,2022
373,1.56,2.350,1.0627,23.19m,9.07m,1,2019-03-31,2019-12-31,2019-05-29,1.5%,5269,2019
8881,10.94,17.650,6.7400,3.08b,438.63m,4,2020-12-31,2020-12-31,2021-02-26,1.6%,1066,2020
8578,2.36,0.000,1.4900,45.75m,4.44m,2,2021-06-30,2021-12-31,2021-08-30,1.6%,8273,2021
417,-2.18,1.420,1.1999,24.56m,-14.91m,4,2021-03-31,2021-03-31,2021-04-20,-1.8%,5120,2021


In [122]:
df2=pd.read_csv("div_quarter_report.csv")

In [123]:
df2

,Unnamed: 0,EPS,DPS,NTA,Revenue,P/L,Quarter,Q Date,Financial Year,Announced,ROE,Stock Code,Year
0,0,1.21,0.0,0.1106,988.21m,13.66m,3,2022-09-30,2022-12-31,2022-11-29,10.9%,5250,2022
1,1,2.29,0.0,0.0984,943.67m,25.77m,2,2022-06-30,2022-12-31,2022-08-25,23.3%,5250,2022
2,2,2.16,0.0,0.1015,839.98m,24.38m,1,2022-03-31,2022-12-31,2022-05-26,21.3%,5250,2022
3,3,2.60,0.0,0.0799,795.06m,29.24m,4,2021-12-31,2021-12-31,2022-02-24,32.5%,5250,2021
4,4,0.13,0.0,0.0522,680.22m,1.44m,3,2021-09-30,2021-12-31,2021-11-25,2.5%,5250,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,11760,2.35,2.0,1.2851,39.45m,10.79m,1,2019-02-28,2019-11-30,2019-04-17,1.8%,5131,2019
11761,11761,3.44,4.0,1.2934,45.03m,15.83m,4,2018-11-30,2018-11-30,2019-01-23,2.7%,5131,2018
11762,11762,3.36,2.0,1.2731,51.02m,17.11m,3,2018-08-31,2018-11-30,2018-10-17,2.6%,5131,2018
11763,11763,2.62,2.0,1.2559,44.02m,12.07m,2,2018-05-31,2018-11-30,2018-07-18,2.1%,5131,2018


In [124]:
del df2['Unnamed: 0']
del df2['Revenue']
del df2['P/L']
del df2['Quarter']
del df2['Q Date']

In [125]:
df2

,EPS,DPS,NTA,Financial Year,Announced,ROE,Stock Code,Year
0,1.21,0.0,0.1106,2022-12-31,2022-11-29,10.9%,5250,2022
1,2.29,0.0,0.0984,2022-12-31,2022-08-25,23.3%,5250,2022
2,2.16,0.0,0.1015,2022-12-31,2022-05-26,21.3%,5250,2022
3,2.60,0.0,0.0799,2021-12-31,2022-02-24,32.5%,5250,2021
4,0.13,0.0,0.0522,2021-12-31,2021-11-25,2.5%,5250,2021
...,...,...,...,...,...,...,...,...
11760,2.35,2.0,1.2851,2019-11-30,2019-04-17,1.8%,5131,2019
11761,3.44,4.0,1.2934,2018-11-30,2019-01-23,2.7%,5131,2018
11762,3.36,2.0,1.2731,2018-11-30,2018-10-17,2.6%,5131,2018
11763,2.62,2.0,1.2559,2018-11-30,2018-07-18,2.1%,5131,2018


In [126]:
df2.dtypes

EPS               float64
DPS               float64
NTA               float64
Financial Year     object
Announced          object
ROE                object
Stock Code         object
Year                int64
dtype: object

In [127]:
df2.rename(columns={"EPS": "quar_eps", "DPS": "quar_dps", "NTA": "quar_nta", "Financial Year": "quar_financial_year", 
                    "Announced": "quar_annouced_date", "ROE": "quar_roe(%)", "Stock Code": "quar_stock_code", "Year": "quar_year"}, inplace=True)

In [129]:
df2['quar_roe(%)'] = df2['quar_roe(%)'].str.slice(0,-1)

In [130]:
df2

,quar_eps,quar_dps,quar_nta,quar_financial_year,quar_annouced_date,quar_roe(%),quar_stock_code,quar_year
0,1.21,0.0,0.1106,2022-12-31,2022-11-29,10.9,5250,2022
1,2.29,0.0,0.0984,2022-12-31,2022-08-25,23.3,5250,2022
2,2.16,0.0,0.1015,2022-12-31,2022-05-26,21.3,5250,2022
3,2.60,0.0,0.0799,2021-12-31,2022-02-24,32.5,5250,2021
4,0.13,0.0,0.0522,2021-12-31,2021-11-25,2.5,5250,2021
...,...,...,...,...,...,...,...,...
11760,2.35,2.0,1.2851,2019-11-30,2019-04-17,1.8,5131,2019
11761,3.44,4.0,1.2934,2018-11-30,2019-01-23,2.7,5131,2018
11762,3.36,2.0,1.2731,2018-11-30,2018-10-17,2.6,5131,2018
11763,2.62,2.0,1.2559,2018-11-30,2018-07-18,2.1,5131,2018


In [131]:
df2['quar_roe(%)'] = df2['quar_roe(%)'].apply(pd.to_numeric, errors='coerce')

In [132]:
df2.dtypes

quar_eps               float64
quar_dps               float64
quar_nta               float64
quar_financial_year     object
quar_annouced_date      object
quar_roe(%)            float64
quar_stock_code         object
quar_year                int64
dtype: object

In [133]:
df2['quar_year'] = df2['quar_year'].astype(str)

In [134]:
df2.dtypes

quar_eps               float64
quar_dps               float64
quar_nta               float64
quar_financial_year     object
quar_annouced_date      object
quar_roe(%)            float64
quar_stock_code         object
quar_year               object
dtype: object

In [135]:
df2['quar_financial_year'] = pd.to_datetime(df2['quar_financial_year'])
df2['quar_financial_year'] = pd.to_datetime(df2['quar_financial_year'].dt.strftime('%Y-%m-%d'))

df2['quar_annouced_date'] = pd.to_datetime(df2['quar_annouced_date'])
df2['quar_annouced_date'] = pd.to_datetime(df2['quar_annouced_date'].dt.strftime('%Y-%m-%d'))

In [136]:
df2.dtypes

quar_eps                      float64
quar_dps                      float64
quar_nta                      float64
quar_financial_year    datetime64[ns]
quar_annouced_date     datetime64[ns]
quar_roe(%)                   float64
quar_stock_code                object
quar_year                      object
dtype: object

In [137]:
df2.to_csv("div_quarter_report(Final).csv")